# Lab Assignment 8: Data Management Using `pandas`, Part 1
## DS 6001: Practice and Application of Data Science

### Instructions
Please answer the following questions as completely as possible using text, code, and the results of code as needed. Format your answers in a Jupyter notebook. To receive full credit, make sure you address every part of the problem, and make sure your document is formatted in a clean and professional way.

In this lab, you will be working with the [2017 Workplace Health in America survey](https://www.cdc.gov/workplacehealthpromotion/survey/data.html) which was conducted by the Centers for Disease Control and Prevention. According to the survey's [guidence document](https://www.cdc.gov/workplacehealthpromotion/data-surveillance/docs/2017-WHA-Guidance-Document-for-Use-of-Public-Data-files-508.pdf):

> The Workplace Health in America (WHA) Survey gathered information from a cross-sectional, nationally representative sample of US worksites. The sample was drawn from the Dun & Bradstreet (D&B) database of all private and public employers in the United States with at least 10 employees. Like previous national surveys, the worksite served as the sampling unit rather than the companies or firms to which the worksites belonged. Worksites were selected using a stratified simple random sample (SRS) design, where the primary strata were ten multi-state regions defined by the Centers for Disease Control and Prevention (CDC), plus an additional stratum containing all hospital worksites.

The data contain over 300 features that report the industry and type of company where the respondents are employed, what kind of health insurance and other health programs are offered, and other characteristics of the workplaces including whether employees are allowed to work from home and the gender and age makeup of the workforce. The data are full of interesting information, but in order to make use of the data a great deal of data manipulation is required first.

## Problem 0
Import the following libraries:

In [187]:
!pip install sidetable

In [188]:
import numpy as np
import pandas as pd
import sidetable
import sqlite3
import warnings
warnings.filterwarnings('ignore')

## Problem 1
The raw data are stored in an ASCII file on the 2017 Workplace Health in America survey [homepage](https://www.cdc.gov/workplacehealthpromotion/survey/data.html). Load the raw data directly into Python without downloading the data onto your harddrive and display a dataframe with only the 14th, 28th, and 102nd rows of the data. [1 point]

In [189]:
# Problem 1
url = "https://www.cdc.gov/workplace-health-promotion/media/files/2024/06/whpps_120717.csv"
data = pd.read_csv(url, sep="~")
selected_rows = data.iloc[[13, 27, 101]]
print(selected_rows)

    OC1  OC3  HI1  HI2  HI3  HI4  HRA1  HRA1A  HRA1B  HRA1E  ...  WL3_05  \
13    3  1.0  2.0  3.0  2.0  1.0   1.0    3.0    3.0    1.0  ...     NaN   
27    1  3.0  1.0  3.0  1.0  1.0   1.0    2.0    4.0    2.0  ...     NaN   
101   2  1.0  1.0  3.0  2.0  1.0   1.0    2.0    4.0    2.0  ...     NaN   

     E1_09  Suppquex       Id  Region  CDC_Region  Industry  Size  Varstrata  \
13     NaN       1.0   1437.0     4.0         6.0       7.0   3.0        0.0   
27     NaN       1.0   2501.0     2.0         4.0       7.0   8.0        0.0   
101    NaN       2.0  12636.0     4.0         6.0       7.0   4.0        0.0   

     Finalwt_worksite,,,,  
13       47.793940929,,,,  
27       47.793940929,,,,  
101      47.793940929,,,,  

[3 rows x 301 columns]


## Problem 2
The data contain 301 columns. Create a new variable in Python's memory to store a working version of the data. In the working version, delete all of the columns except for the following:

* `Industry`: 7 Industry Categories with NAICS codes

* `Size`: 8 Employee Size Categories

* `OC3` Is your organization for profit, non-profit, government?

* `HI1` In general, do you offer full, partial or no payment of premiums for personal health insurance for full-time employees?

* `HI2` Over the past 12 months, were full-time employees asked to pay a larger proportion, smaller proportion or the same proportion of personal health insurance premiums?

* `HI3`: Does your organization offer personal health insurance for your part-time employees?

* `CP1`: Are there health education programs, which focus on skill development and lifestyle behavior change along with information dissemination and awareness building?

* `WL6`: Allow employees to work from home?

* Every column that begins `WD`, expressing the percentage of employees that have certain characteristics at the firm

[1 point]

In [190]:
# Problem 2
wd_columns = [col for col in data.columns if col.startswith('WD')]
selected_columns = ['Industry', 'Size', 'OC3', 'HI1', 'HI2', 'HI3', 'CP1', 'WL5'] + wd_columns
working_data = data[selected_columns]

In [191]:
working_data

,Industry,Size,OC3,HI1,HI2,HI3,CP1,WL5,WD1_1,WD1_2,WD2,WD3,WD4,WD5,WD6,WD7
0,7.0,7.0,3.0,2.0,1.0,2.0,1.0,1.0,25.0,20.0,85.0,60.0,40.0,15.0,0.0,22.0
1,7.0,6.0,3.0,2.0,3.0,1.0,1.0,1.0,997.0,997.0,90.0,90.0,997.0,997.0,0.0,997.0
2,7.0,8.0,3.0,1.0,3.0,1.0,1.0,1.0,35.0,4.0,997.0,997.0,40.0,15.0,997.0,997.0
3,7.0,4.0,2.0,1.0,2.0,1.0,2.0,1.0,50.0,15.0,50.0,85.0,75.0,0.0,0.0,997.0
4,7.0,4.0,3.0,1.0,3.0,1.0,1.0,1.0,50.0,40.0,60.0,60.0,40.0,30.0,0.0,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2838,6.0,5.0,4.0,1.0,3.0,1.0,1.0,99.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0
2839,6.0,5.0,4.0,2.0,3.0,1.0,1.0,2.0,997.0,997.0,997.0,997.0,997.0,997.0,997.0,997.0
2840,6.0,8.0,4.0,2.0,3.0,1.0,1.0,1.0,27.0,997.0,61.0,997.0,997.0,997.0,997.0,997.0
2841,6.0,8.0,4.0,2.0,3.0,1.0,2.0,99.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0


## Problem 3
The [codebook](https://www.cdc.gov/workplacehealthpromotion/data-surveillance/docs/2017-WHA-Datafile-Codebook-508.pdf) for the WHA data contain short descriptions of the meaning of each of the columns in the data. Use these descriptions to decide on better and more intuitive names for the columns in the working version of the data, and rename the columns accordingly. [1 point]

In [192]:
rename_dict = {
    'Industry': 'Industry_Category',
    'Size': 'Employee_Size_Category',
    'OC3': 'Organization_Type',
    'HI1': 'Premium_Payment_FT',
    'HI2': 'Premium_Proportion_Change_FT',
    'HI3': 'Insurance_Offered_PT',
    'CP1': 'Health_Education_Programs',
    'WL5': 'Work_From_Home_Policy',
    'WD1_1': 'Pct_Employees_Under_30',
    'WD1_2': 'Pct_Employees_60_Plus',
    'WD2': 'Pct_Female_Employees',
    'WD3': 'Pct_Hourly_Workers',
    'WD4': 'Pct_Non_Daytime_Shift',
    'WD5': 'Pct_Remote_Workers',
    'WD6': 'Pct_Unionized',
    'WD7': 'Annual_Turnover_Pct'
}

In [193]:
# Rename the columns in working_data
working_data = working_data.rename(columns=rename_dict)

In [194]:
working_data.head()

,Industry_Category,Employee_Size_Category,Organization_Type,Premium_Payment_FT,Premium_Proportion_Change_FT,Insurance_Offered_PT,Health_Education_Programs,Work_From_Home_Policy,Pct_Employees_Under_30,Pct_Employees_60_Plus,Pct_Female_Employees,Pct_Hourly_Workers,Pct_Non_Daytime_Shift,Pct_Remote_Workers,Pct_Unionized,Annual_Turnover_Pct
0,7.0,7.0,3.0,2.0,1.0,2.0,1.0,1.0,25.0,20.0,85.0,60.0,40.0,15.0,0.0,22.0
1,7.0,6.0,3.0,2.0,3.0,1.0,1.0,1.0,997.0,997.0,90.0,90.0,997.0,997.0,0.0,997.0
2,7.0,8.0,3.0,1.0,3.0,1.0,1.0,1.0,35.0,4.0,997.0,997.0,40.0,15.0,997.0,997.0
3,7.0,4.0,2.0,1.0,2.0,1.0,2.0,1.0,50.0,15.0,50.0,85.0,75.0,0.0,0.0,997.0
4,7.0,4.0,3.0,1.0,3.0,1.0,1.0,1.0,50.0,40.0,60.0,60.0,40.0,30.0,0.0,28.0


In [195]:
print(working_data.columns)

Index(['Industry_Category', 'Employee_Size_Category', 'Organization_Type',
       'Premium_Payment_FT', 'Premium_Proportion_Change_FT',
       'Insurance_Offered_PT', 'Health_Education_Programs',
       'Work_From_Home_Policy', 'Pct_Employees_Under_30',
       'Pct_Employees_60_Plus', 'Pct_Female_Employees', 'Pct_Hourly_Workers',
       'Pct_Non_Daytime_Shift', 'Pct_Remote_Workers', 'Pct_Unionized',
       'Annual_Turnover_Pct'],
      dtype='object')


## Problem 4
Using the codebook and this [dictionary of NAICS industrial codes](https://www.naics.com/search-naics-codes-by-industry/), place descriptive labels on the categories of the industry column in the working data. [1 point]

In [196]:
# Define the mapping dictionary
industry_labels = {
    1: "Agriculture, Mining, Utilities, Construction, Manufacturing",
    2: "Wholesale, Retail, Transportation",
    3: "Arts, Entertainment, Recreation, Accommodation, Food Services, Other Services",
    4: "Information, Finance, Real Estate, Professional Services, Management, Administrative",
    5: "Education, Health Care (excluding hospitals)",
    6: "Public Administration",
    7: "Hospitals"
}

# Apply the mapping to the Industry_Category column
working_data["Industry_Category"] = working_data["Industry_Category"].map(industry_labels)

# Verify the transformation
print(working_data["Industry_Category"].unique())

['Hospitals' 'Agriculture, Mining, Utilities, Construction, Manufacturing'
 'Wholesale, Retail, Transportation'
 'Arts, Entertainment, Recreation, Accommodation, Food Services, Other Services'
 'Information, Finance, Real Estate, Professional Services, Management, Administrative'
 'Education, Health Care (excluding hospitals)' 'Public Administration'
 nan]


In [197]:
working_data.head()

,Industry_Category,Employee_Size_Category,Organization_Type,Premium_Payment_FT,Premium_Proportion_Change_FT,Insurance_Offered_PT,Health_Education_Programs,Work_From_Home_Policy,Pct_Employees_Under_30,Pct_Employees_60_Plus,Pct_Female_Employees,Pct_Hourly_Workers,Pct_Non_Daytime_Shift,Pct_Remote_Workers,Pct_Unionized,Annual_Turnover_Pct
0,Hospitals,7.0,3.0,2.0,1.0,2.0,1.0,1.0,25.0,20.0,85.0,60.0,40.0,15.0,0.0,22.0
1,Hospitals,6.0,3.0,2.0,3.0,1.0,1.0,1.0,997.0,997.0,90.0,90.0,997.0,997.0,0.0,997.0
2,Hospitals,8.0,3.0,1.0,3.0,1.0,1.0,1.0,35.0,4.0,997.0,997.0,40.0,15.0,997.0,997.0
3,Hospitals,4.0,2.0,1.0,2.0,1.0,2.0,1.0,50.0,15.0,50.0,85.0,75.0,0.0,0.0,997.0
4,Hospitals,4.0,3.0,1.0,3.0,1.0,1.0,1.0,50.0,40.0,60.0,60.0,40.0,30.0,0.0,28.0


## Problem 5
Using the codebook, recode the "size" column to have three categories: "Small" for workplaces with fewer than 100 employees, "Medium" for workplaces with at least 100 but fewer than 500 employees, and "Large" for companies with at least 500 employees. [Note: Python dataframes have an attribute `.size` that reports the space the dataframe takes up in memory. Don't confuse this attribute with the column named "Size" in the raw data.] [1 point]

In [198]:
working_data["Size_Group"] = working_data["Employee_Size_Category"].map(size_mapping)

In [199]:
working_data["Size_Group"]

,Size_Group
0,Large
1,Large
2,Large
3,Medium
4,Medium
...,...
2838,Medium
2839,Medium
2840,Large
2841,Large


In [200]:
print(working_data["Employee_Size_Category"].unique())

[ 7.  6.  8.  4.  1.  3.  5.  2. nan]


In [201]:
missing_count = working_data["Size_Group"].isna().sum()
print(f"Number of missing values: {missing_count}")

Number of missing values: 1


In [202]:
size_mapping = {
    1.0: "Small",
    2.0: "Small",
    3.0: "Small",
    4.0: "Medium",
    5.0: "Medium",
    6.0: "Large",
    7.0: "Large",
    8.0: "Large"
}

# Apply the mapping
working_data["Size_Group"] = working_data["Employee_Size_Category"].map(size_mapping)

# Verify the recoding
print("\nSize Group Counts:")
print(working_data["Size_Group"].value_counts())


Size Group Counts:
Size_Group
Small     2195
Medium     393
Large      254
Name: count, dtype: int64


In [203]:
working_data.head()

,Industry_Category,Employee_Size_Category,Organization_Type,Premium_Payment_FT,Premium_Proportion_Change_FT,Insurance_Offered_PT,Health_Education_Programs,Work_From_Home_Policy,Pct_Employees_Under_30,Pct_Employees_60_Plus,Pct_Female_Employees,Pct_Hourly_Workers,Pct_Non_Daytime_Shift,Pct_Remote_Workers,Pct_Unionized,Annual_Turnover_Pct,Size_Group
0,Hospitals,7.0,3.0,2.0,1.0,2.0,1.0,1.0,25.0,20.0,85.0,60.0,40.0,15.0,0.0,22.0,Large
1,Hospitals,6.0,3.0,2.0,3.0,1.0,1.0,1.0,997.0,997.0,90.0,90.0,997.0,997.0,0.0,997.0,Large
2,Hospitals,8.0,3.0,1.0,3.0,1.0,1.0,1.0,35.0,4.0,997.0,997.0,40.0,15.0,997.0,997.0,Large
3,Hospitals,4.0,2.0,1.0,2.0,1.0,2.0,1.0,50.0,15.0,50.0,85.0,75.0,0.0,0.0,997.0,Medium
4,Hospitals,4.0,3.0,1.0,3.0,1.0,1.0,1.0,50.0,40.0,60.0,60.0,40.0,30.0,0.0,28.0,Medium


## Problem 6
Use the codebook to write accurate and descriptive labels for each category for each categorical column in the working data. Then apply all of these labels to the data at once. Code "Legitimate Skip", "Don't know", "Refused", and "Blank" as missing values. [2 points]

In [204]:
# Define all mapping dictionaries
industry_labels = {
    1: "Agriculture, Mining, Utilities, Construction, Manufacturing (NAICS 11, 21, 22, 23, 31-33)",
    2: "Wholesale, Retail, Transportation (NAICS 42, 44-45, 48-49)",
    3: "Arts, Entertainment, Accommodation, Food Services, Other Services (NAICS 71, 72, 81)",
    4: "Information, Finance, Insurance, Real Estate, Professional Services (NAICS 51, 52, 53, 54, 55, 56)",
    5: "Education, Health Care (excluding hospitals) (NAICS 61, 62)",
    6: "Public Administration (NAICS 92)",
    7: "Hospital Worksites (NAICS 622110, 622210, 622310)"
}

size_labels = {
    1: "10-24 employees",
    2: "25-49 employees",
    3: "50-99 employees",
    4: "100-249 employees",
    5: "250-499 employees",
    6: "500-749 employees",
    7: "750-999 employees",
    8: "1,000+ employees"
}

org_type_labels = {
    1: "For-profit",
    2: "Non-profit",
    3: "Government"
}

premium_payment_labels = {
    1: "Full insurance coverage offered",
    2: "Partial insurance coverage offered",
    3: "No insurance coverage offered"
}

premium_change_labels = {
    1: "Larger proportion",
    2: "Smaller proportion",
    3: "About the same"
}

insurance_pt_labels = {
    1: "Yes",
    2: "No"
}

health_edu_labels = {
    1: "Yes",
    2: "No"
}

# Define missing codes
missing_codes = [96, 97, 98, 99]

# List of categorical columns with their mappings
categorical_columns = {
    'Organization_Type': org_type_labels,
    'Premium_Payment_FT': premium_payment_labels,
    'Premium_Proportion_Change_FT': premium_change_labels,
    'Insurance_Offered_PT': insurance_pt_labels,
    'Health_Education_Programs': health_edu_labels
}

# Apply labels and handle missing values
for col, mapping in categorical_columns.items():
    # Replace missing codes with NaN
    working_data[col] = working_data[col].replace(missing_codes, np.nan)
    # Apply descriptive labels
    working_data[col] = working_data[col].map(mapping)

In [205]:
working_data.head()

,Industry_Category,Employee_Size_Category,Organization_Type,Premium_Payment_FT,Premium_Proportion_Change_FT,Insurance_Offered_PT,Health_Education_Programs,Work_From_Home_Policy,Pct_Employees_Under_30,Pct_Employees_60_Plus,Pct_Female_Employees,Pct_Hourly_Workers,Pct_Non_Daytime_Shift,Pct_Remote_Workers,Pct_Unionized,Annual_Turnover_Pct,Size_Group
0,Hospitals,7.0,Government,Partial insurance coverage offered,Larger proportion,No,Yes,1.0,25.0,20.0,85.0,60.0,40.0,15.0,0.0,22.0,Large
1,Hospitals,6.0,Government,Partial insurance coverage offered,About the same,Yes,Yes,1.0,997.0,997.0,90.0,90.0,997.0,997.0,0.0,997.0,Large
2,Hospitals,8.0,Government,Full insurance coverage offered,About the same,Yes,Yes,1.0,35.0,4.0,997.0,997.0,40.0,15.0,997.0,997.0,Large
3,Hospitals,4.0,Non-profit,Full insurance coverage offered,Smaller proportion,Yes,No,1.0,50.0,15.0,50.0,85.0,75.0,0.0,0.0,997.0,Medium
4,Hospitals,4.0,Government,Full insurance coverage offered,About the same,Yes,Yes,1.0,50.0,40.0,60.0,60.0,40.0,30.0,0.0,28.0,Medium


## Problem 7
The features that measure the percent of the workforce with a particular characteristic use the codes 997, 998, and 999 to represent "Don't know", "Refusal", and "Blank/Invalid" respectively. Replace these values with missing values for all of the percentage features at the same time. [1 point]

In [206]:
# List of percentage columns
percentage_columns = [
    'Pct_Employees_Under_30',
    'Pct_Employees_60_Plus',
    'Pct_Female_Employees',
    'Pct_Hourly_Workers',
    'Pct_Non_Daytime_Shift',
    'Pct_Remote_Workers',
    'Pct_Unionized',
    'Annual_Turnover_Pct'
]

# Codes to replace
codes_to_replace = [997, 998, 999]

# Replace codes with NaN in all percentage columns simultaneously
working_data[percentage_columns] = working_data[percentage_columns].replace(codes_to_replace, np.nan)

In [207]:
working_data.head()

,Industry_Category,Employee_Size_Category,Organization_Type,Premium_Payment_FT,Premium_Proportion_Change_FT,Insurance_Offered_PT,Health_Education_Programs,Work_From_Home_Policy,Pct_Employees_Under_30,Pct_Employees_60_Plus,Pct_Female_Employees,Pct_Hourly_Workers,Pct_Non_Daytime_Shift,Pct_Remote_Workers,Pct_Unionized,Annual_Turnover_Pct,Size_Group
0,Hospitals,7.0,Government,Partial insurance coverage offered,Larger proportion,No,Yes,1.0,25.0,20.0,85.0,60.0,40.0,15.0,0.0,22.0,Large
1,Hospitals,6.0,Government,Partial insurance coverage offered,About the same,Yes,Yes,1.0,NaN,NaN,90.0,90.0,NaN,NaN,0.0,NaN,Large
2,Hospitals,8.0,Government,Full insurance coverage offered,About the same,Yes,Yes,1.0,35.0,4.0,NaN,NaN,40.0,15.0,NaN,NaN,Large
3,Hospitals,4.0,Non-profit,Full insurance coverage offered,Smaller proportion,Yes,No,1.0,50.0,15.0,50.0,85.0,75.0,0.0,0.0,NaN,Medium
4,Hospitals,4.0,Government,Full insurance coverage offered,About the same,Yes,Yes,1.0,50.0,40.0,60.0,60.0,40.0,30.0,0.0,28.0,Medium


## Problem 8
Sort the working data by industry in ascending alphabetical order. Within industry categories, sort the rows by size in ascending alphabetical order. Within groups with the same industry and size, sort by percent of the workforce that is under 30 in descending numeric order. [1 point]

In [208]:
# Sort the DataFrame
working_data = working_data.sort_values(
    by=['Industry_Category', 'Employee_Size_Category', 'Pct_Employees_Under_30'],
    ascending=[True, True, False]
)

In [209]:
working_data.head()

,Industry_Category,Employee_Size_Category,Organization_Type,Premium_Payment_FT,Premium_Proportion_Change_FT,Insurance_Offered_PT,Health_Education_Programs,Work_From_Home_Policy,Pct_Employees_Under_30,Pct_Employees_60_Plus,Pct_Female_Employees,Pct_Hourly_Workers,Pct_Non_Daytime_Shift,Pct_Remote_Workers,Pct_Unionized,Annual_Turnover_Pct,Size_Group
900,"Agriculture, Mining, Utilities, Construction, ...",1.0,NaN,No insurance coverage offered,NaN,No,No,1.0,100.0,0.0,90.0,90.0,1.0,1.0,0.0,80.0,Small
2034,"Agriculture, Mining, Utilities, Construction, ...",1.0,Non-profit,NaN,NaN,No,No,1.0,100.0,0.0,40.0,95.0,100.0,0.0,0.0,NaN,Small
2051,"Agriculture, Mining, Utilities, Construction, ...",1.0,Non-profit,Full insurance coverage offered,About the same,No,No,1.0,95.0,1.0,1.0,50.0,50.0,5.0,0.0,30.0,Small
1188,"Agriculture, Mining, Utilities, Construction, ...",1.0,Non-profit,Partial insurance coverage offered,About the same,No,No,2.0,80.0,15.0,2.0,80.0,0.0,0.0,0.0,2.0,Small
1180,"Agriculture, Mining, Utilities, Construction, ...",1.0,Non-profit,Full insurance coverage offered,About the same,No,No,1.0,75.0,1.0,5.0,5.0,0.0,75.0,0.0,5.0,Small


## Problem 9
There is one row in the working data that has a `NaN` value for industry. Delete this row. Use a logical expression, and not the row number. [1 point]

In [210]:
# Drop any row with a null value in 'Industry_Category'
working_data = working_data.dropna(subset=['Industry_Category'])

# Verify the result
working_data.head()

,Industry_Category,Employee_Size_Category,Organization_Type,Premium_Payment_FT,Premium_Proportion_Change_FT,Insurance_Offered_PT,Health_Education_Programs,Work_From_Home_Policy,Pct_Employees_Under_30,Pct_Employees_60_Plus,Pct_Female_Employees,Pct_Hourly_Workers,Pct_Non_Daytime_Shift,Pct_Remote_Workers,Pct_Unionized,Annual_Turnover_Pct,Size_Group
900,"Agriculture, Mining, Utilities, Construction, ...",1.0,NaN,No insurance coverage offered,NaN,No,No,1.0,100.0,0.0,90.0,90.0,1.0,1.0,0.0,80.0,Small
2034,"Agriculture, Mining, Utilities, Construction, ...",1.0,Non-profit,NaN,NaN,No,No,1.0,100.0,0.0,40.0,95.0,100.0,0.0,0.0,NaN,Small
2051,"Agriculture, Mining, Utilities, Construction, ...",1.0,Non-profit,Full insurance coverage offered,About the same,No,No,1.0,95.0,1.0,1.0,50.0,50.0,5.0,0.0,30.0,Small
1188,"Agriculture, Mining, Utilities, Construction, ...",1.0,Non-profit,Partial insurance coverage offered,About the same,No,No,2.0,80.0,15.0,2.0,80.0,0.0,0.0,0.0,2.0,Small
1180,"Agriculture, Mining, Utilities, Construction, ...",1.0,Non-profit,Full insurance coverage offered,About the same,No,No,1.0,75.0,1.0,5.0,5.0,0.0,75.0,0.0,5.0,Small


## Problem 10
Create a new feature named `gender_balance` that has three categories: "Mostly men" for workplaces with between 0% and 35% female employees, "Balanced" for workplaces with more than 35% and at most 65% female employees, and "Mostly women" for workplaces with more than 65% female employees. [1 point]

In [211]:
# Define bins and labels
bins = [0, 35, 65, 100]
labels = ['Mostly men', 'Balanced', 'Mostly women']

# Create the 'gender_balance' column
working_data['gender_balance'] = pd.cut(
    working_data['Pct_Female_Employees'],
    bins=bins,
    labels=labels,
    include_lowest=True
)

## Problem 11
Change the data type of all categorical features in the working data from "object" to "category". [1 point]

In [212]:
# List of categorical columns
categorical_columns = [
    'Industry_Category',
    'Employee_Size_Category',
    'Organization_Type',
    'Premium_Payment_FT',
    'Premium_Proportion_Change_FT',
    'Insurance_Offered_PT',
    'Health_Education_Programs',
    'Work_From_Home_Policy',
    'Size_Group',
    'gender_balance'
]

# Convert each column to 'category' type
for col in categorical_columns:
    working_data[col] = working_data[col].astype('category')

# Verify the changes
print(working_data.dtypes)

Industry_Category               category
Employee_Size_Category          category
Organization_Type               category
Premium_Payment_FT              category
Premium_Proportion_Change_FT    category
Insurance_Offered_PT            category
Health_Education_Programs       category
Work_From_Home_Policy           category
Pct_Employees_Under_30           float64
Pct_Employees_60_Plus            float64
Pct_Female_Employees             float64
Pct_Hourly_Workers               float64
Pct_Non_Daytime_Shift            float64
Pct_Remote_Workers               float64
Pct_Unionized                    float64
Annual_Turnover_Pct              float64
Size_Group                      category
gender_balance                  category
dtype: object


## Problem 12
Filter the data to only those rows that represent small workplaces that allow employees to work from home. Then report how many of these workplaces offer full insurance, partial insurance, and no insurance. Use a function that reports the percent, cumulative count, and cumulative percent in addition to the counts. [1 point]

In [213]:
# Filter the data
filtered_data = working_data[
    (working_data['Size_Group'] == 'Small') &
    (working_data['Work_From_Home_Policy'] == 1.0)
]

# Calculate counts and percentages for insurance offerings
insurance_counts = filtered_data['Premium_Payment_FT'].value_counts()
insurance_percent = filtered_data['Premium_Payment_FT'].value_counts(normalize=True) * 100

# Calculate cumulative counts and percentages
cumulative_counts = insurance_counts.cumsum()
cumulative_percent = insurance_percent.cumsum()

# Create a report DataFrame
report = pd.DataFrame({
    'Count': insurance_counts,
    'Percent': insurance_percent,
    'Cumulative Count': cumulative_counts,
    'Cumulative Percent': cumulative_percent
})

# Display the report
print("Insurance Offerings for Small Workplaces Allowing Work from Home:")
print(report)

Insurance Offerings for Small Workplaces Allowing Work from Home:
                                    Count    Percent  Cumulative Count  \
Premium_Payment_FT                                                       
Partial insurance coverage offered    459  43.714286               459   
Full insurance coverage offered       418  39.809524               877   
No insurance coverage offered         173  16.476190              1050   

                                    Cumulative Percent  
Premium_Payment_FT                                      
Partial insurance coverage offered           43.714286  
Full insurance coverage offered              83.523810  
No insurance coverage offered               100.000000  


## Problem 13
Anything that can be done in SQL can be done with `pandas`. The next several questions ask you to write `pandas` code to match a given SQL query. But to check that the SQL query and `pandas` code yield the same result, create a new database wsing the `sqlite3` package and input the cleaned WHA data as a table in this database. (See module 6 for a discussion of SQlite in Python.) [1 point]

In [214]:
# Connect to a new SQLite database (creates the file if it doesn’t exist)
conn = sqlite3.connect('working_data.db')

# Write the DataFrame to a table in the database
working_data.to_sql('working_table', conn, if_exists='replace', index=False)

# Close the connection
conn.close()

# Optional verification: Check the first few rows
conn = sqlite3.connect('working_data.db')
print(pd.read_sql("SELECT * FROM working_table LIMIT 5", conn))
conn.close()

                                   Industry_Category Employee_Size_Category  \
0  Agriculture, Mining, Utilities, Construction, ...                    1.0   
1  Agriculture, Mining, Utilities, Construction, ...                    1.0   
2  Agriculture, Mining, Utilities, Construction, ...                    1.0   
3  Agriculture, Mining, Utilities, Construction, ...                    1.0   
4  Agriculture, Mining, Utilities, Construction, ...                    1.0   

  Organization_Type                  Premium_Payment_FT  \
0              None       No insurance coverage offered   
1        Non-profit                                None   
2        Non-profit     Full insurance coverage offered   
3        Non-profit  Partial insurance coverage offered   
4        Non-profit     Full insurance coverage offered   

  Premium_Proportion_Change_FT Insurance_Offered_PT Health_Education_Programs  \
0                         None                   No                        No   
1       

In [215]:
working_data

,Industry_Category,Employee_Size_Category,Organization_Type,Premium_Payment_FT,Premium_Proportion_Change_FT,Insurance_Offered_PT,Health_Education_Programs,Work_From_Home_Policy,Pct_Employees_Under_30,Pct_Employees_60_Plus,Pct_Female_Employees,Pct_Hourly_Workers,Pct_Non_Daytime_Shift,Pct_Remote_Workers,Pct_Unionized,Annual_Turnover_Pct,Size_Group,gender_balance
900,"Agriculture, Mining, Utilities, Construction, ...",1.0,NaN,No insurance coverage offered,NaN,No,No,1.0,100.0,0.0,90.0,90.0,1.0,1.0,0.0,80.0,Small,Mostly women
2034,"Agriculture, Mining, Utilities, Construction, ...",1.0,Non-profit,NaN,NaN,No,No,1.0,100.0,0.0,40.0,95.0,100.0,0.0,0.0,NaN,Small,Balanced
2051,"Agriculture, Mining, Utilities, Construction, ...",1.0,Non-profit,Full insurance coverage offered,About the same,No,No,1.0,95.0,1.0,1.0,50.0,50.0,5.0,0.0,30.0,Small,Mostly men
1188,"Agriculture, Mining, Utilities, Construction, ...",1.0,Non-profit,Partial insurance coverage offered,About the same,No,No,2.0,80.0,15.0,2.0,80.0,0.0,0.0,0.0,2.0,Small,Mostly men
1180,"Agriculture, Mining, Utilities, Construction, ...",1.0,Non-profit,Full insurance coverage offered,About the same,No,No,1.0,75.0,1.0,5.0,5.0,0.0,75.0,0.0,5.0,Small,Mostly men
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1271,"Wholesale, Retail, Transportation",8.0,Government,Full insurance coverage offered,About the same,Yes,Yes,99.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Large,NaN
1513,"Wholesale, Retail, Transportation",8.0,Non-profit,Partial insurance coverage offered,About the same,No,Yes,97.0,NaN,NaN,NaN,NaN,40.0,NaN,0.0,NaN,Large,NaN
1514,"Wholesale, Retail, Transportation",8.0,For-profit,Full insurance coverage offered,NaN,No,Yes,1.0,NaN,NaN,NaN,NaN,60.0,NaN,NaN,NaN,Large,NaN
1775,"Wholesale, Retail, Transportation",8.0,Non-profit,Full insurance coverage offered,About the same,No,Yes,2.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,22.0,Large,NaN


## Problem 14
Write `pandas` code that replicates the output of the following SQL code:
```
SELECT size, type, premiums AS insurance, percent_female FROM whpps
WHERE industry = 'Hospitals' AND premium_change='Smaller'
ORDER BY percent_female DESC;
```
For each of these queries, your feature names might be different from the ones listed in the query, depending on the names you chose in problem 3.
[2 points]

In [216]:
# Filter the DataFrame
filtered_data = working_data[
    (working_data['Industry_Category'] == 'Hospitals') &
    (working_data['Premium_Proportion_Change_FT'] == 'Smaller proportion')
]

# Select and rename columns
result = filtered_data[['Employee_Size_Category', 'Organization_Type', 'Premium_Payment_FT', 'Pct_Female_Employees']].rename(
    columns={
        'Employee_Size_Category': 'size',
        'Organization_Type': 'type',
        'Premium_Payment_FT': 'insurance',
        'Pct_Female_Employees': 'percent_female'
    }
)

# Sort by percent_female in descending order
result = result.sort_values(by='percent_female', ascending=False)

In [217]:
result

,size,type,insurance,percent_female
320,4.0,Government,Full insurance coverage offered,89.0
187,8.0,Government,Partial insurance coverage offered,80.0
214,8.0,Government,Partial insurance coverage offered,80.0
229,2.0,Government,Full insurance coverage offered,75.0
191,5.0,Government,Partial insurance coverage offered,65.0
3,4.0,Non-profit,Full insurance coverage offered,50.0
11,4.0,NaN,Partial insurance coverage offered,NaN
75,5.0,Government,Full insurance coverage offered,NaN
48,5.0,Government,Partial insurance coverage offered,NaN
51,5.0,Government,Full insurance coverage offered,NaN


## Problem 15
Write `pandas` code that replicates the output of the following SQL code:
```
SELECT industry,
    AVG(percent_female) as percent_female,
    AVG(percent_under30) as percent_under30,
    AVG(percent_over60) as percent_over60
FROM whpps
GROUP BY industry
ORDER BY percent_female DESC;
```
[2 points]

In [218]:
# Group by Industry_Category and calculate means
grouped_data = working_data.groupby('Industry_Category').agg(
    percent_female=('Pct_Female_Employees', 'mean'),
    percent_under30=('Pct_Employees_Under_30', 'mean'),
    percent_over60=('Pct_Employees_60_Plus', 'mean')
).reset_index()

# Rename Industry_Category to industry
grouped_data = grouped_data.rename(columns={'Industry_Category': 'industry'})

# Sort by percent_female in descending order
result = grouped_data.sort_values(by='percent_female', ascending=False)

In [219]:
result

,industry,percent_female,percent_under30,percent_over60
2,"Education, Health Care (excluding hospitals)",80.657143,25.745665,11.349570
3,Hospitals,76.427027,27.213793,16.489655
1,"Arts, Entertainment, Recreation, Accommodation...",53.804416,38.566343,11.544872
4,"Information, Finance, Real Estate, Professiona...",50.632184,23.821752,12.465465
5,Public Administration,39.056738,21.015625,15.015385
6,"Wholesale, Retail, Transportation",32.657258,29.108696,12.584034
0,"Agriculture, Mining, Utilities, Construction, ...",20.328605,22.257143,10.690355


## Problem 16
Write `pandas` code that replicates the output of the following SQL code:
```
SELECT gender_balance, premiums, COUNT(*)
FROM whpps
GROUP BY gender_balance, premiums
HAVING gender_balance is NOT NULL and premiums is NOT NULL;
```
[2 points]

In [220]:
# Filter out rows where gender_balance or Premium_Payment_FT is null
filtered_data = working_data.dropna(subset=['gender_balance', 'Premium_Payment_FT'])

# Group by gender_balance and Premium_Payment_FT, count rows
result = filtered_data.groupby(['gender_balance', 'Premium_Payment_FT']).size().reset_index(name='count')

# Rename Premium_Payment_FT to premiums
result = result.rename(columns={'Premium_Payment_FT': 'premiums'})

In [221]:
result

,gender_balance,premiums,count
0,Mostly men,Full insurance coverage offered,301
1,Mostly men,No insurance coverage offered,91
2,Mostly men,Partial insurance coverage offered,332
3,Balanced,Full insurance coverage offered,226
4,Balanced,No insurance coverage offered,77
5,Balanced,Partial insurance coverage offered,271
6,Mostly women,Full insurance coverage offered,267
7,Mostly women,No insurance coverage offered,107
8,Mostly women,Partial insurance coverage offered,333
